In [1]:
from sklearn.model_selection import KFold
import pandas as pd

In [2]:
import wandb
wandb.login()

wandb: Currently logged in as: konggedzu (imucs). Use `wandb login --relogin` to force relogin


True

In [3]:
sweep_config = {
    'method': 'random'
}

In [4]:
metric = {
    'name': 'Grand Mean',
    'goal': 'maximize'
}

sweep_config['metric'] = metric

In [5]:
parameters_dict = {
    'dropout': {
          'values': [0.1]
        },

    'learning_rate': {
        'values': [0.0005]
        },

    'batch_size': {
        'values': [4096]
        },

    'data_augmentation_multiple': {
        'values': [5]
        }
}

In [6]:
sweep_config['parameters'] = parameters_dict

In [7]:
sweep_id = wandb.sweep(sweep_config, project="pytorch-sweeps-sub_loc_train")

Create sweep with ID: e9g4dl9v
Sweep URL: https://wandb.ai/imucs/pytorch-sweeps-sub_loc_train/sweeps/e9g4dl9v


In [8]:
import pprint

pprint.pprint(sweep_config)

{'method': 'random',
 'metric': {'goal': 'maximize', 'name': 'Grand Mean'},
 'parameters': {'batch_size': {'values': [4096]},
                'data_augmentation_multiple': {'values': [5]},
                'dropout': {'values': [0.1]},
                'learning_rate': {'values': [0.0005]}}}


In [9]:
feature_pd = pd.read_csv('/home/kongge/projects/new_protT5/data/DPC_T5_578_right.csv')
labels_pd = pd.read_csv("/home/kongge/projects/new_protT5/data/mutil_label_578.csv")

In [10]:
from dataAug.tools import MLDA

In [11]:
from dataAug.all_tools import dataAugSMOTE

In [12]:

smote_multiple = {}

In [13]:
import time
from classify.targeTools import testThresholdFive, Accuracy
from torch import optim
from torch.utils.data import DataLoader, TensorDataset
import torch
from Classify_adjust import ModelClassify
def train_and_val(config=None):
    with wandb.init(config=config):
        config = wandb.config
        kf = KFold(n_splits=10, shuffle=True)



        model_discord = []
        for train_index, test_index in kf.split(feature_pd):

            train_true_data = feature_pd.iloc[train_index]
            train_true_label = labels_pd.iloc[train_index]

            multi_label_samples = train_true_label[(train_true_label.sum(axis=1) >= 2)]
            multi_label_indices = multi_label_samples.index
            multi_features_samples = train_true_data.loc[multi_label_indices]
            ML_G_X, ML_G_y = MLDA(multi_features_samples, multi_label_samples, config.data_augmentation_multiple)

            G_feature, G_label = dataAugSMOTE(train_true_data, train_true_label, config.data_augmentation_multiple, 1424)
            G_feature = pd.concat([G_feature, ML_G_X], axis=0)
            G_label = pd.concat([G_label, ML_G_y], axis=0)

            train_feature = pd.concat([train_true_data, G_feature], axis=0)
            train_label = pd.concat([train_true_label, G_label], axis=0)

            test_data = feature_pd.iloc[test_index]
            test_label = labels_pd.iloc[test_index]

            datasetTrain = TensorDataset(torch.tensor(train_feature.values), torch.tensor(train_label.values))
            batch_size = config.batch_size
            dataloaderTrain = DataLoader(datasetTrain, batch_size=batch_size, shuffle=True)

            datasetTest = TensorDataset(torch.tensor(test_data.values), torch.tensor(test_label.values))

            batch_size = len(datasetTest)
            dataloaderTest = DataLoader(datasetTest, batch_size=batch_size, shuffle=False)

            model = ModelClassify(drop_rate=config.dropout, num_class=5, feature_num=1424)
            criterion = torch.nn.BCELoss()
            optimizer = optim.Adam(model.parameters(), lr=config.learning_rate)
            import warnings
            warnings.filterwarnings("ignore")
            GMList = {}
            epochs = 150
            for epoch in range(epochs):
                model.train()
                total_loss = 0.0
                for idx, data in enumerate(dataloaderTrain, 0):
                    inputs, labels = data
                    labels = labels.float()
                    inputs = inputs.float()
                    out = model(inputs)
                    loss = criterion(out, labels)
                    optimizer.zero_grad()
                    loss.backward()
                    optimizer.step()
                    total_loss += loss.item()
                avg_loss = total_loss / len(dataloaderTrain)
                threshold = 0.5
                labels_cov = torch.where(out > threshold, torch.tensor(1), torch.tensor(0))
                print(f"Epoch [{epoch+1}/{epochs}], Average Loss: {avg_loss:.4f}, ACC: {Accuracy(labels.int(), labels_cov)}")
            GMScore1 = testThresholdFive(epoch, model, dataloaderTest, class_num=5)
            GMList[epoch] = GMScore1
            t = time.time()
            best_key = max(GMList, key=GMList.get)
            best_value = GMList[best_key]
            best_value = [x.item() if isinstance(x, torch.Tensor) else x for x in best_value]
            model_discord.append(best_value)
        model_discord_column_means = [sum(col) / len(col) for col in zip(*model_discord)]
        smote_multiple[int(t)] = model_discord_column_means
        wandb.log({"Grand Mean": model_discord_column_means[0]})

In [14]:
wandb.agent(sweep_id, train_and_val, count=10)

wandb: Agent Starting Run: ww3u6ify with config:
wandb: 	batch_size: 4096
wandb: 	data_augmentation_multiple: 5
wandb: 	dropout: 0.1
wandb: 	learning_rate: 0.0005


/home/kongge/.conda/envs/kongge_pytorch_cpu/lib/python3.7/site-packages/imblearn/utils/_validation.py:314: UserWarning: After over-sampling, the number of samples (1068) in class 0 will be larger than the number of samples in the majority class (class #1 -> 342)
  f"After over-sampling, the number of samples ({n_samples})"
/home/kongge/.conda/envs/kongge_pytorch_cpu/lib/python3.7/site-packages/imblearn/utils/_validation.py:314: UserWarning: After over-sampling, the number of samples (570) in class 0 will be larger than the number of samples in the majority class (class #1 -> 425)
  f"After over-sampling, the number of samples ({n_samples})"
/home/kongge/.conda/envs/kongge_pytorch_cpu/lib/python3.7/site-packages/imblearn/utils/_validation.py:314: UserWarning: After over-sampling, the number of samples (1266) in class 0 will be larger than the number of samples in the majority class (class #1 -> 309)
  f"After over-sampling, the number of samples ({n_samples})"


Epoch [1/150], Average Loss: 0.7348, ACC: 0.19143996013951206
Epoch [2/150], Average Loss: 0.5491, ACC: 0.48199800697558115
Epoch [3/150], Average Loss: 0.4781, ACC: 0.5519631290483266
Epoch [4/150], Average Loss: 0.4363, ACC: 0.6187892376681579
Epoch [5/150], Average Loss: 0.4011, ACC: 0.6769058295964108
Epoch [6/150], Average Loss: 0.3735, ACC: 0.7163677130044848
Epoch [7/150], Average Loss: 0.3498, ACC: 0.7533881415047337
Epoch [8/150], Average Loss: 0.3303, ACC: 0.776781265570504
Epoch [9/150], Average Loss: 0.3149, ACC: 0.7986796213253614
Epoch [10/150], Average Loss: 0.2992, ACC: 0.8289486796213255
Epoch [11/150], Average Loss: 0.2873, ACC: 0.8498256103637271
Epoch [12/150], Average Loss: 0.2759, ACC: 0.870029895366219
Epoch [13/150], Average Loss: 0.2669, ACC: 0.8897608370702546
Epoch [14/150], Average Loss: 0.2578, ACC: 0.9057797708021921
Epoch [15/150], Average Loss: 0.2503, ACC: 0.9207772795216743
Epoch [16/150], Average Loss: 0.2435, ACC: 0.9281016442451419
Epoch [17/150], A

Grand Mean,▁
Grand Mean,0.81937


wandb: Agent Starting Run: 115hahyt with config:
wandb: 	batch_size: 4096
wandb: 	data_augmentation_multiple: 5
wandb: 	dropout: 0.1
wandb: 	learning_rate: 0.0005


...

Epoch [1/150], Average Loss: 0.7402, ACC: 0.19112280701754414
Epoch [2/150], Average Loss: 0.5364, ACC: 0.48519799498746435
Epoch [3/150], Average Loss: 0.4713, ACC: 0.5850275689223019
Epoch [4/150], Average Loss: 0.4293, ACC: 0.6442807017543833
Epoch [5/150], Average Loss: 0.3953, ACC: 0.6894436090225549
Epoch [6/150], Average Loss: 0.3696, ACC: 0.7328320802005004
Epoch [7/150], Average Loss: 0.3455, ACC: 0.7663408521303258
Epoch [8/150], Average Loss: 0.3258, ACC: 0.7976190476190478
Epoch [9/150], Average Loss: 0.3097, ACC: 0.8288771929824567
Epoch [10/150], Average Loss: 0.2952, ACC: 0.8509523809523819
Epoch [11/150], Average Loss: 0.2828, ACC: 0.8708020050125312
Epoch [12/150], Average Loss: 0.2718, ACC: 0.8911779448621556
Epoch [13/150], Average Loss: 0.2610, ACC: 0.9155187969924813
Epoch [14/150], Average Loss: 0.2528, ACC: 0.9202756892230575
Epoch [15/150], Average Loss: 0.2448, ACC: 0.9334636591478699
Epoch [16/150], Average Loss: 0.2375, ACC: 0.9442606516290728
Epoch [17/150],

Grand Mean,▁
Grand Mean,0.83575


wandb: Agent Starting Run: 6wxmasom with config:
wandb: 	batch_size: 4096
wandb: 	data_augmentation_multiple: 5
wandb: 	dropout: 0.1
wandb: 	learning_rate: 0.0005


Epoch [1/150], Average Loss: 0.7438, ACC: 0.2016542102640764
Epoch [2/150], Average Loss: 0.5625, ACC: 0.4483856502242113
Epoch [3/150], Average Loss: 0.4951, ACC: 0.5411160936721443
Epoch [4/150], Average Loss: 0.4490, ACC: 0.6108769307423977
Epoch [5/150], Average Loss: 0.4148, ACC: 0.6558794220229176
Epoch [6/150], Average Loss: 0.3870, ACC: 0.7033134030891877
Epoch [7/150], Average Loss: 0.3633, ACC: 0.738390632785253
Epoch [8/150], Average Loss: 0.3430, ACC: 0.7685600398604885
Epoch [9/150], Average Loss: 0.3249, ACC: 0.7965620328849033
Epoch [10/150], Average Loss: 0.3092, ACC: 0.8211011459890388
Epoch [11/150], Average Loss: 0.2961, ACC: 0.8449177877428997
Epoch [12/150], Average Loss: 0.2844, ACC: 0.8634280019930248
Epoch [13/150], Average Loss: 0.2734, ACC: 0.8823368211260585
Epoch [14/150], Average Loss: 0.2636, ACC: 0.895864474339811
Epoch [15/150], Average Loss: 0.2545, ACC: 0.9126806178375688
Epoch [16/150], Average Loss: 0.2469, ACC: 0.9206278026905832
Epoch [17/150], Ave

Grand Mean,▁
Grand Mean,0.83852


In [15]:
smote_multiple

{1695265023: [0.8193732619285583,
  0.7260738052026618,
  0.8129738858640854,
  0.8213173031806946,
  0.8938294112682342,
  0.842671936750412],
 1695265954: [0.8193162083625793,
  0.737144585601936,
  0.8136569872958258,
  0.8244857847690582,
  0.8812109231948853,
  0.8400827288627625],
 1695266894: [0.8303621768951416,
  0.7578644888082274,
  0.8255999193385763,
  0.8362522721290588,
  0.8835501134395599,
  0.8485441207885742],
 1695267854: [0.8204002916812897,
  0.7563218390804598,
  0.8159104658197217,
  0.8245613992214202,
  0.8688596606254577,
  0.8363480806350708],
 1695268634: [0.8337477326393128,
  0.7492740471869327,
  0.8279290179471669,
  0.8383393824100495,
  0.8984724640846252,
  0.8547237753868103],
 1695269268: [0.8070036172866821,
  0.7062008469449486,
  0.8003478523895946,
  0.8110102951526642,
  0.8852692127227784,
  0.8321899890899658],
 1695269869: [0.8121607422828674,
  0.7113127646702964,
  0.8049052228271829,
  0.8150030314922333,
  0.8921355068683624,
  0.837447

In [46]:
import json
file = open("/home/kongge/projects/new_protT5/data/dictionary_data.json", "w")
json.dump(smote_multiple, file)
file.close()